In [1]:
import numpy as np
import pandas as pd

from transformers import GPTNeoForCausalLM, GPT2Tokenizer



/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# prompt options:
# Joyce Beatty
# Joyce Beatty says the following:
# Joyce Beatty says:

In [ ]:
'''gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=300,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]'''

'''import numpy as np
tokenizer.decode(np.argmax(outputs.logits.detach().numpy(),axis=2)[0])'''

In [2]:
# read whole corpus
processed_df = pd.read_pickle('../data/processed_df.pkl')

# work on subset of speakers
temp_bioguides = ['H001055','C001074']
temp_df = processed_df[processed_df.bioguide_id.isin(temp_bioguides)]

In [15]:
temp_row = temp_df.iloc[0]

prompt = temp_row['first_name'] + ' ' + temp_row['last_name'] + ' ' + temp_row['speech']

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.gpt_neo = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
        self.tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

    def forward(self, x):
        
        inputs = self.tokenizer(x, return_tensors="pt")
        outputs = self.gpt_neo(**inputs, labels=inputs.input_ids)
        
        return outputs.loss,outputs.logits


net = Net()

optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [24]:
for epoch in range(10):  # loop over the dataset multiple times

    #for i, data in enumerate(trainloader, 0):

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    loss,logits = net(prompt)
    loss.backward()
    optimizer.step()

    print(loss.item())

2.62884783744812
1.0245797634124756
0.5738399624824524
0.27172327041625977
0.13085439801216125
0.06838947534561157
0.023092877119779587
0.004251773934811354
0.002531480509787798
0.001702785142697394


In [30]:
inputs = net.tokenizer(temp_row['first_name'] + ' ' + temp_row['last_name'], return_tensors="pt")
gen_tokens = net.gpt_neo.generate(
    inputs.input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=300,
)
gen_text = net.tokenizer.batch_decode(gen_tokens)[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [43]:
net.tokenizer.pad_token = net.tokenizer.eos_token

In [44]:
net.tokenizer([temp_row['first_name'] + ' ' + temp_row['last_name'],temp_row['first_name']],padding=True,return_tensors='tf')

2022-04-28 15:32:27.167877: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


{'input_ids': <tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[   51, 16956,  5932,   364],
       [   51, 16956, 50256, 50256]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [1, 1, 0, 0]], dtype=int32)>}

In [ ]:
# show it works for temporary data, then report it quickly